# Transit Corridor Improvements

## Criteria
* Transit routes that intersect the SHN (at least 35% of the route is within a 0.5 mile buffer of the SHN)
* average route speed is below **12 mph**  **AND**
* **10%+ of its trips are competitive** against a car 
    * The car travels the same path as the bus (same stops along the way).
    * A bus trip is competitive if it is within 1.5x the car's travel time (bus can take 50% longer).
    * Across all the trips taken by that route, if **at least 10% trips** are competitive, the route is deemed competitive. 
    * Avoid selecting 0% trips competitive, which gets at circulator/loop type routes

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import branca 
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import Markdown, HTML

from shared_utils import calitp_color_palette as cp
from shared_utils import geography_utils

catalog = intake.open_catalog("./*.yml")

CUSTOM_CATEGORICAL = [
    cp.CALITP_CATEGORY_BRIGHT_COLORS[2], # yellow
    cp.CALITP_CATEGORY_BRIGHT_COLORS[1], # orange
    cp.CALITP_CATEGORY_BRIGHT_COLORS[0], # blue
    cp.CALITP_CATEGORY_BRIGHT_COLORS[5], # purple  
    cp.CALITP_CATEGORY_BRIGHT_COLORS[3], # green
]

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

In [ ]:
gdf = catalog.bus_routes_all_aggregated_stats.read().to_crs(geography_utils.WGS84)

include = ["intersects_shn"]
gdf = gdf[gdf.category == "intersects_shn"].reset_index(drop=True)

In [ ]:
METRIC_CUTOFFS = {
    # show routes with average speeds <= 12 mph
    "mean_speed_mph": 12, 
    # show routes with >= 10% of trips being competitive against a car 
    # competitive is if it takes bus no longer than +20% longer than a car
    "pct_trips_competitive": 0.10,
}

In [ ]:
#def set_cutoffs(gdf: gpd.GeoDataFrame, 
#                speed_threshold: int, 
#                pct_competitive_threshold: float,
#               ) -> gpd.GeoDataFrame: 
#    
#    print(f"speed cutoff: {speed_threshold}, pct competitive cutoff: {pct_competitive_threshold}")
#    # Get statewide counts to see how many fall into each district
#    subset = gdf[(gdf.mean_speed_mph <= speed_threshold) &
#                 (gdf.pct_trips_competitive >= pct_competitive_threshold)
#                ]
#    print(f"# obs statewide: {len(subset)}")
#    display(subset.caltrans_district.value_counts())

In [ ]:
#for speeds in [10, 12, 15]:
#    for pct_competitive in [0.05, 0.10, 0.15]:
#        set_cutoffs(gdf, speeds, pct_competitive)

In [ ]:
def subset_to_district_sort_metric(gdf: gpd.GeoDataFrame, 
                                   district: str,
                                   speed_cutoff: int = 12,
                                   pct_competitive_cutoff: float = 0.05
                                  ): 
    keep_cols = ["calitp_itp_id", "route_id", 
            "mean_speed_mph", "pct_trips_competitive",
            "category", "geometry"]
    
    df = (gdf[(gdf.caltrans_district == district) & 
              (gdf.mean_speed_mph <= speed_cutoff) & 
              (gdf.pct_trips_competitive >= pct_competitive_cutoff)
             ]
          .reset_index(drop=True)
          [keep_cols]
         )
    
    df2 = df.assign(
        mean_speed_mph = df.mean_speed_mph.round(2),
        pct_trips_competitive = df.pct_trips_competitive.round(3)
    )
               
    return df2

In [ ]:
TILES = "CartoDB Positron" 

def make_map(gdf: gpd.GeoDataFrame, district: str):        
    m = gdf.explore("mean_speed_mph", categorical = False, 
                    cmap = ZERO_THIRTY_COLORSCALE, tiles = TILES)
    
    display(m)
    
    m2 = gdf.explore("route_id", categorical = True, 
                    cmap = "tab20", tiles = TILES)
    
    display(m2)
    
    display(gdf.sort_values(
        ["mean_speed_mph", "pct_trips_competitive"], 
        ascending=[True, True]).drop(columns = ["geometry", "category"]))

## District Maps

In [ ]:
districts = gdf[gdf.caltrans_district.notna()].caltrans_district.unique().tolist()

for d in sorted(districts):
    display(HTML(f"<h3> District {d}</h3>"))

    subset = subset_to_district_sort_metric(
        gdf, d, 
        speed_cutoff = METRIC_CUTOFFS["mean_speed_mph"],
        pct_competitive_cutoff = METRIC_CUTOFFS["pct_trips_competitive"]
    )

    if len(subset) > 0:
        make_map(subset, d)
    else:
        print("No routes meet this criteria")